In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import codecs
import csv
import os
import datetime

### Resample raw data

In [42]:
def get_resample_df(df, freq):
    
    df = df[df['EX'] == 'N']
    df['DATE'] = df['DATE'].astype('str')
    df['TIME_M'] = df['DATE'] + ' ' + df['TIME_M']
    df['TIME_M'] = pd.to_datetime(df['TIME_M'], format = '%Y%m%d %H:%M:%S.%f')
    df['SIZE'] = df['SIZE'].astype(int)
    df['PRICE'] = df['PRICE'].astype(float)
    df['TRADES'] = 1 # indicate number of trades
    
    resample_df = df.resample(f'{freq}min', on='TIME_M', label = 'right').agg( # "right" stands for in 9:30-9:33, take 9:33 as label
        {'PRICE': ['first', 'max', 'min', 'last'], 
         'SIZE': 'sum', 
         'TRADES': 'sum',
         'SYM_ROOT': 'first'})
    resample_df.columns = resample_df.columns.droplevel(0)
    resample_df.columns = ['open', 'high', 'low', 'close', 'volume', 'trades', 'symbol']
    
    # drop overnight
    resample_df = resample_df.between_time('09:31', '16:00')
    # drop weekends, holidays
    resample_df['date'] = resample_df.index.date
    resample_df['total_trades'] = resample_df.groupby(['date'])['trades'].transform('sum')
    resample_df = resample_df[resample_df['total_trades'] > 0]
    
    return resample_df

In [43]:
def generate_resample_df_monthly(file_name, freq):
    os.mkdir(f'{freq}min data')
    csv_storage = []
    symbol_list = ['A'] # initialize the symbol list with the first symbol in the csv file
    with codecs.open(f'{file_name}.csv', 'r', encoding='utf-8') as fp:
        fp_key = csv.reader(fp)
        for csv_key in fp_key:
            csv_reader = csv.DictReader(fp, fieldnames=csv_key)
            
            for row in csv_reader:
                symbol = row['SYM_ROOT']
                if symbol in symbol_list: # which means we don't iterate to the next symbol
                    csv_storage.append(row)
                else:
                    prev_symbol = symbol_list[-1]
                    resample_df = get_resample_df(pd.DataFrame(csv_storage), freq) # get the previous symbol's resample df
                    if len(resample_df) == 0:
                        print('no data for', prev_symbol)
                    else:
                        to_file = f'{freq}min data/{prev_symbol} {freq}min {file_name} .csv'
                        resample_df.to_csv(to_file)
                    
                    symbol_list.append(symbol) # add the new symbol to the list
                    csv_storage = [] # refresh the csv_storage
                    csv_storage.append(row) # add the new symbol's first row to the csv_storage

                

In [ ]:
generate_resample_df_monthly(file_name = "2013-01", freq = 1)

### Labeling

In [ ]:
total_label_df = pd.DataFrame(columns=['Date', 'Overnight Return', 'Intraday Return', 'Daily Return', 'Symbol'])
csv_storage = []
symbol_list = ['A'] # initialize the symbol list with the first symbol in the csv file
with codecs.open('2013-09.csv', 'r', encoding='utf-8') as fp:
    fp_key = csv.reader(fp)
    for csv_key in fp_key:
        csv_reader = csv.DictReader(fp, fieldnames=csv_key)
        
        for row in csv_reader:
            symbol = row['SYM_ROOT']
            if symbol in symbol_list: # which means we don't iterate to the next symbol
                csv_storage.append(row)
            else:
                prev_symbol = symbol_list[-1]
                label_df = get_label_df(pd.DataFrame(csv_storage)) # get the previous symbol's resample df
                if len(label_df) == 0:
                    print('no data for', prev_symbol)
                else:
                    total_label_df = total_label_df.append(label_df)
                
                symbol_list.append(symbol) # add the new symbol to the list
                csv_storage = [] # refresh the csv_storage
                csv_storage.append(row) # add the new symbol's first row to the csv_storage

In [18]:
total_label_df = total_label_df.set_index('Date')
total_label_df.to_csv('label.csv')

In [16]:
total_label_df.iloc[50:100]

,Date,Overnight Return,Intraday Return,Daily Return,Symbol
12,20130919,0.003182,-0.032429,-0.029349,AAN
13,20130920,-0.014572,-0.005545,-0.020036,AAN
14,20130923,-0.002230,0.005589,0.003346,AAN
15,20130924,-0.001482,0.012987,0.011486,AAN
16,20130925,-0.001099,0.012468,0.011355,AAN
17,20130926,-0.005071,0.005096,0.000000,AAN
18,20130927,-0.003622,0.006180,0.002535,AAN
0,20130903,0.000627,-0.001002,-0.000376,AAP
1,20130904,-0.000752,0.003889,0.003134,AAP
2,20130905,0.000250,-0.011744,-0.011497,AAP


In [2]:
def get_label_df(df):
    df = df[df['EX'] == 'N']
    df['DATE'] = df['DATE'].astype('str')
    df['TIME_M'] = df['DATE'] + ' ' + df['TIME_M']
    df['TIME_M'] = pd.to_datetime(df['TIME_M'], format = '%Y%m%d %H:%M:%S.%f')
    df = df.set_index('TIME_M')
    df = df.between_time('09:30', '16:00')
    df['PRICE'] = df['PRICE'].astype(float)
    
    df_label = pd.DataFrame(columns=['Date', 'Overnight Return', 'Intraday Return', 'Daily Return', 'Symbol'])
    date_list = df['DATE'].unique()
    for today, tomorrow in zip(date_list, date_list[1:]):
        close_price_today = df[df['DATE'] == today]['PRICE'].iloc[-1] # last price of today at 16:00
        open_price_tomorrow = df[df['DATE'] == tomorrow]['PRICE'].iloc[0] # first price of tomorrow at 9:30
        close_price_tomorrow = df[df['DATE'] == tomorrow]['PRICE'].iloc[-1] # last price of tomorrow at 16:00
        overnight_return = (open_price_tomorrow - close_price_today) / close_price_today
        intraday_return = (close_price_tomorrow - open_price_tomorrow) / open_price_tomorrow
        daily_return = (close_price_tomorrow - close_price_today) / close_price_today
        symbol = df[df['DATE'] == today]['SYM_ROOT'].iloc[0]  
        df_label = df_label.append({'Date': today, 
                                    'Overnight Return': overnight_return, 
                                    'Intraday Return': intraday_return, 
                                    'Daily Return': daily_return, 
                                    'Symbol': symbol}, ignore_index=True)
        
    return df_label
    

In [2]:
df_test = pd.read_csv(r"C:\Users\hzixu\Desktop\201309lite.csv")
df_test = df_test[df_test['EX'] == 'N']
df_test['DATE'] = df_test['DATE'].astype('str')
df_test['TIME_M'] = df_test['DATE'] + ' ' + df_test['TIME_M']
df_test['TIME_M'] = pd.to_datetime(df_test['TIME_M'], format = '%Y%m%d %H:%M:%S.%f')
df_test = df_test.set_index('TIME_M')
df_test = df_test.between_time('09:30', '16:00')
df_test['PRICE'] = df_test['PRICE'].astype(float)
df_test

,DATE,EX,SYM_ROOT,SYM_SUFFIX,TR_SCOND,SIZE,PRICE,TR_STOPIND,TR_CORR,TR_SEQNUM,TR_SOURCE,TR_RF
TIME_M,,,,,,,,,,,,
2013-09-03 09:30:00.013,20130903,N,A,NaN,O,18653,47.04,N,0,3670,C,NaN
2013-09-03 09:30:00.702,20130903,N,A,NaN,@,100,47.11,N,0,4133,C,NaN
2013-09-03 09:30:01.004,20130903,N,A,NaN,F,100,47.04,N,0,4216,C,NaN
2013-09-03 09:30:01.491,20130903,N,A,NaN,@,100,47.11,N,0,4377,C,NaN
2013-09-03 09:30:01.493,20130903,N,A,NaN,@,179,47.11,N,0,4378,C,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2013-09-04 14:45:55.884,20130904,N,A,NaN,@,100,47.79,N,0,921783,C,NaN
2013-09-04 14:45:55.885,20130904,N,A,NaN,@,200,47.79,N,0,921788,C,NaN
2013-09-04 14:45:58.071,20130904,N,A,NaN,@,300,47.81,N,0,921996,C,NaN


In [4]:
df_label = pd.DataFrame(columns=['Date', 'Overnight Return', 'Intraday Return', 'Daily Return', 'Symbol'])
date_list = df_test['DATE'].unique()
for today, tomorrow in zip(date_list, date_list[1:]):
    close_price_today = df_test[df_test['DATE'] == today]['PRICE'].iloc[-1] # last price of today at 16:00
    open_price_tomorrow = df_test[df_test['DATE'] == tomorrow]['PRICE'].iloc[0] # first price of tomorrow at 9:30
    close_price_tomorrow = df_test[df_test['DATE'] == tomorrow]['PRICE'].iloc[-1] # last price of tomorrow at 16:00
    overnight_return = (open_price_tomorrow - close_price_today) / close_price_today
    intraday_return = (close_price_tomorrow - open_price_tomorrow) / open_price_tomorrow
    daily_return = (close_price_tomorrow - close_price_today) / close_price_today
    symbol = df_test[df_test['DATE'] == today]['SYM_ROOT'].iloc[0]  
    df_label = df_label.append({'Date': today, 
                                'Overnight Return': overnight_return, 
                                'Intraday Return': intraday_return, 
                                'Daily Return': daily_return, 
                                'Symbol': symbol}, ignore_index=True)
    
df_label

C:\Users\hzixu\AppData\Local\Temp/ipykernel_8592/143251904.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_label = df_label.append({'Date': today,


,Date,Overnight Return,Intraday Return,Daily Return,Symbol
0,20130903,0.000426,0.018751,0.019186,A


### Comparison

In [45]:
file_name = "C:\\Users\\hzixu\\Desktop\\testing\\1min data\\"
file_list = os.listdir(file_name)
symbol_list = []
for file in file_list:
    symbol_list.append(file.split(" ")[0])

symbol_list

['A',
 'AA',
 'AAN',
 'AAP',
 'AAT',
 'AAV',
 'AB',
 'ABB',
 'ABBV',
 'ABC',
 'ABG',
 'ABM',
 'ABR',
 'ABT',
 'ABV',
 'ABW',
 'ABX',
 'ACC',
 'ACCO',
 'ACE',
 'ACG',
 'ACH',
 'ACI',
 'ACM',
 'ACMP',
 'ACN',
 'ACO',
 'ACP',
 'ACRE',
 'ACT',
 'ACTV',
 'ACW',
 'ADC',
 'ADM',
 'ADS',
 'ADT',
 'ADX',
 'ADY',
 'AEB',
 'AEC',
 'AED',
 'AEE',
 'AEF',
 'AEG',
 'AEH',
 'AEK',
 'AEL',
 'AEM',
 'AEO',
 'AEP',
 'AER',
 'AES',
 'AET',
 'AEV',
 'AF',
 'AFA',
 'AFB',
 'AFC',
 'AFF',
 'AFG',
 'AFL',
 'AFM',
 'AFQ',
 'AFSD',
 'AFT',
 'AFW',
 'AG',
 'AGC',
 'AGCO',
 'AGD',
 'AGM',
 'AGN',
 'AGO',
 'AGRO',
 'AGU',
 'AH',
 'AHC',
 'AHL',
 'AHS',
 'AHT',
 'AI',
 'AIB',
 'AIG',
 'AIN',
 'AIQ',
 'AIR',
 'AIT',
 'AIV',
 'AIZ',
 'AJG',
 'AKO',
 'AKP',
 'AKR',
 'AKS',
 'AL',
 'ALB',
 'ALC',
 'ALDW',
 'ALE',
 'ALEX',
 'ALG',
 'ALJ',
 'ALK',
 'ALL',
 'ALLY',
 'ALP',
 'ALR',
 'ALSN',
 'ALU',
 'ALV',
 'ALX',
 'AM',
 'AMBO',
 'AMD',
 'AME',
 'AMG',
 'AMID',
 'AMP',
 'AMRC',
 'AMRE',
 'AMT',
 'AMTD',
 'AMTG',
 'AMX',


In [46]:
diff_list = []

for symbol in symbol_list:    
    df_self = pd.read_csv(rf"C:\Users\hzixu\Desktop\testing\1min data\{symbol} 1min 2013-01 .csv")
    df_self['date'] = df_self['date'].astype('str')
    df_self = df_self[df_self['date'] == '2013-01-03']
    df_self.reset_index(drop=True, inplace=True)
    df_self = df_self.set_index('TIME_M')
    df_self = df_self.drop(columns=['date', 'total_trades', 'symbol'])
    df_self = df_self.astype(float)
    df_self = df_self.fillna(0)

    df_sample = pd.read_csv(r"C:\Users\hzixu\Desktop\testing\20231025\TAQ-20130103.csv")
    df_sample = df_sample[df_sample['SYM_ROOT'] == symbol]
    df_sample['DATE'] = df_sample['DATE'].astype('int').astype('str')
    df_sample['hour'] = df_sample['hour'].astype('int').astype('str')
    df_sample['min'] = df_sample['min'].astype('int').astype('str')
    df_sample['TIME_M'] = df_sample['DATE'] + ' ' + df_sample['hour'] + ':' + df_sample['min']
    df_sample['TIME_M'] = pd.to_datetime(df_sample['TIME_M'], format = '%Y%m%d %H:%M')
    df_sample = df_sample.set_index('TIME_M')
    df_sample = df_sample.drop(['DATE', 'hour', 'min', 'SYM_ROOT', 'SYM_SUFFIX'], axis=1)
    df_sample = df_sample.astype('float')
    df_sample = df_sample.rename(columns={'count': 'trades'})
    df_sample = df_sample[['open', 'high', 'low', 'close', 'volume', 'trades']]
    df_sample = df_sample.reindex(pd.date_range(start='2013-01-03 09:30:00', end='2013-01-03 15:59:00', freq='1min'), fill_value=0)
    df_sample.index = df_sample.index + datetime.timedelta(minutes=1)

 
    for i in df_self.index:
    # if i not in df_sample.index:
    #     print(i, 'not exist in df_sample')
    # else:
        diff = df_self.loc[i] - df_sample.loc[i]
        if diff.any() != 0:
            diff = diff.append(pd.Series(i)).append(pd.Series(symbol))
            diff_list.append(diff)
        else:
            # print('same data for', i) 
            pass

C:\Users\hzixu\AppData\Local\Temp/ipykernel_22108/1406603704.py:35: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  diff = diff.append(pd.Series(i)).append(pd.Series(symbol))
C:\Users\hzixu\AppData\Local\Temp/ipykernel_22108/1406603704.py:35: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  diff = diff.append(pd.Series(i)).append(pd.Series(symbol))
C:\Users\hzixu\AppData\Local\Temp/ipykernel_22108/1406603704.py:35: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  diff = diff.append(pd.Series(i)).append(pd.Series(symbol))
C:\Users\hzixu\AppData\Local\Temp/ipykernel_22108/1406603704.py:35: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  diff = 

In [47]:
diff_df = pd.concat(diff_list, axis=1).T
diff_df.columns = ['diff_open', 'diff_high', 'diff_low', 'diff_close', 'diff_volume', 'diff_trades', 'time', 'symbol']
diff_df.to_csv('comparison 2013-01-03.csv')
diff_df

,diff_open,diff_high,diff_low,diff_close,diff_volume,diff_trades,time,symbol
0,0.06,0.0,0.0,0.0,31262.0,1.0,2013-01-03 09:31:00,A
1,0.01,0.0,0.0,0.0,202515.0,1.0,2013-01-03 09:31:00,AA
2,-0.06,0.0,0.0,0.0,2906.0,1.0,2013-01-03 09:31:00,AAN
3,0.72,0.72,0.0,0.0,5057.0,1.0,2013-01-03 09:31:00,AAP
4,28.62,28.62,28.62,28.62,1028.0,1.0,2013-01-03 09:31:00,AAT
...,...,...,...,...,...,...,...,...
652,0.0,0.0,0.0,0.0,32966.0,1.0,2013-01-03 09:31:00,AZN
653,-0.55,0.0,0.0,0.0,3088.0,1.0,2013-01-03 09:31:00,AZO
654,-0.19,0.0,-0.19,0.0,1814.0,1.0,2013-01-03 09:31:00,AZZ
655,0.06,0.0,0.0,0.0,3329.0,1.0,2013-01-03 09:33:00,B
